In [ ]:
#Look for a model to be trained to perform semantic segmentation
#Dataset + related guide: https://idiotdeveloper.com/unet-implementation-in-tensorflow-using-keras-api/
#                         https://www.kaggle.com/datasets/b9d4e32be2f57c2901fc9c5cd5f6633be7075f4b32d73348a6d5db245f2c1934

#Semantic segmentation using U-Net: https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47

In [1]:
pwd()

'/Users/salvo/BackgroundRemover'